**PROJECT OUTCOMES:**
* Create interactive graphs of athlete's change in lean mass index and sum of 7 skinfolds over time from the raw anthropometric data given. 
* Option to have all data included on one graph but with the ability to filter through individual athletes.

In [26]:
# Import necessary modules
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, HoverTool, DatetimeTickFormatter

In [27]:
# Import anthropometric data from desktop as a pandas DataFrame
raw_data = pd.read_csv('/Users/deannetaillieu/desktop/Anthropometric Data.csv', header=0)

In [28]:
# Create new DataFrame with the columns needed to plot lean mass index and sum of 7 skinfolds 
raw_data_crop = raw_data.iloc[:, [0, 1, 3, 4, 5, 6, 7, 9, 10, 11]]

In [29]:
# Remove athletes with less than 2 measurements from the DataFrame
duplicates = raw_data_crop.duplicated(subset=['ID'], keep=False) # Create boolean array that indicates athletes with 2+ measurements
inverted = ~duplicates # Invert the series so that athlete's with 1 measurement are shown as 'True'
non_duplicates_index = [i for i, val in enumerate(inverted) if val] # Find the index of the athletes with only 1 measurement
duplicates_index = [i for i, val in enumerate(duplicates) if val] # Find the index of the athletes with 2+ measurements
anthro_data = raw_data_crop.iloc[duplicates_index, :] # Create new DataFrame for athletes with 2+ measurements
anthro_data = anthro_data.reset_index() # Reset the index of the DataFrame
anthro_data = anthro_data.drop(columns='index') # Remove the old index column
anthro_data['Date'] = pd.to_datetime(anthro_data['Date']) # Convert 'Date' column to datetime format for plotting

In [30]:
# Calculate lean mass index (weight / (sum of 7 skinfolds **0.14)) and sum of 7 skinfolds (triceps, subscap, biceps, illiac, abdomen, thigh, and calf)
anthro_data['Lean Mass Index'] = anthro_data['Weight'] / ((anthro_data['Triceps (mm)'] + anthro_data['SubScap (mm)'] + anthro_data['Biceps (mm)'] + anthro_data['Illiac (mm)'] + anthro_data['Abdomen (mm)'] + anthro_data['Thigh (mm)'] + anthro_data['Calf (mm)']) ** 0.14)
anthro_data['Sum of 7'] = anthro_data['Triceps (mm)'] + anthro_data['SubScap (mm)'] + anthro_data['Biceps (mm)'] + anthro_data['Illiac (mm)'] + anthro_data['Abdomen (mm)'] + anthro_data['Thigh (mm)'] + anthro_data['Calf (mm)']

The equations used above to calculate lean mass index and sum of 7 skinfolds were adapted from:

Slater, G. J., Duthie, G. M., Pyne, D. B., & Hopkins, W. G. (2006). Validation of a skinfold based index for tracking proportional changes in lean mass. British Journal of Sports Medicine, 40(3), 208-213. doi:10.1136/bjsm.2005.019794

In [31]:
# Plot lean mass index for all athletes
data = ColumnDataSource(data=anthro_data)

p1 = figure(plot_width=475, 
            plot_height=475,
            x_axis_type='datetime', 
            x_axis_label='Measurement Date', 
            y_axis_label='Lean Mass Index (mm.kg^−0.14)', 
            tools="", 
            toolbar_location=None)
p1.circle(x='Date', 
          y='Lean Mass Index', 
          color='salmon', 
          alpha=0.7, 
          size=10, 
          source=data)
p1.title.text = 'Lean Mass Index of All Athletes'
p1.title.align = "center"
p1.title.text_color = "salmon"
p1.title.text_font_size = "20px"
p1.title.background_fill_color = "white"
p1.title.text_font='helvetica'
p1.title.text_alpha=0.8
p1.xaxis.formatter = DatetimeTickFormatter(months="%B %Y", years="%B %Y")
p1.xaxis.major_label_orientation = 3/4

# Add HoverTool showing exact lean mass index and date values
hover = HoverTool(tooltips=[('Athlete ID', '@ID'), 
                            ('Lean Mass Index', '@{Lean Mass Index}'), 
                            ('Date', '@Date{%F}')], 
                  formatters={'@Date':'datetime'})
p1.add_tools(hover)

# Plot sum of 7 skinfolds for all athletes
p2 = figure(plot_width=475, 
            plot_height=475, 
            x_axis_type='datetime',
            x_axis_label='Measurement Date',
            y_axis_label='Sum of 7 Skinfolds (mm)',
            tools="", 
            toolbar_location=None)
p2.circle(x='Date', 
          y='Sum of 7',
          color='cadetblue', 
          alpha=0.7, 
          size=10, 
          source=data)
p2.title.text = 'Sum of 7 Skinfolds of All Athletes'
p2.title.align = "center"
p2.title.text_color = "cadetblue"
p2.title.text_font_size = "20px"
p2.title.background_fill_color = "white"
p2.title.text_font='helvetica'
p2.title.text_alpha=0.8
p2.xaxis.formatter = DatetimeTickFormatter(months="%B %Y", years="%B %Y")
p2.xaxis.major_label_orientation = 3/4

# Add HoverTool showing exact sum of 7 and date values
hover = HoverTool(tooltips=[('Athlete ID', '@ID'), 
                            ('Sum of 7 Skinfolds', '@{Sum of 7}'), 
                            ('Date', '@Date{%F}')], 
                  formatters={'@Date':'datetime'})
p2.add_tools(hover)

# Alter layout so graphs appear side by side in a row
layout = row(p1, p2)
output_notebook()
show(layout)

Loading BokehJS ...

In [32]:
# Define the function 'athlete' that graphs lean mass index and sum of 7 skinfolds for individual athletes using Bokeh
def athlete(ID):
    
    """When athlete ID is inputted into the function, produce two interactive graphs: lean mass index and sum of 7 skinfolds."""
    
    # Gather rows where ID column matches the ID inputted along with all columns associated with those rows and sort by date
    athlete_rows = anthro_data.loc[anthro_data['ID'] == ID, :]
    athlete_rows = athlete_rows.sort_values('Date')
    
    # Create DataFrame and Column Data Source with columns 'ID', 'Date', 'Lean Mass Index' and 'Sum of 7'
    athlete_data = athlete_rows[['ID','Date', 'Lean Mass Index', 'Sum of 7']]
    data = ColumnDataSource(data=athlete_data)
    
    # Plot change in lean mass index over time 
    p3 = figure(plot_width=475, 
                plot_height=475, 
                x_axis_type='datetime', 
                x_axis_label='Measurement Date', 
                y_axis_label='Lean Mass Index (mm.kg^−0.14)', 
                tools="", 
                toolbar_location=None)
    p3.line(x='Date', 
            y='Lean Mass Index', 
            line_width=2, 
            color='lightslategray', 
            alpha=0.5, 
            line_dash=[5,5], 
            source=data)
    p3.circle(x='Date', 
              y='Lean Mass Index', 
              size=15, 
              color='salmon', 
              alpha=0.8, 
              source=data)
    p3.title.text = f'Lean Mass Index of Athlete {ID}'
    p3.title.align = "center"
    p3.title.text_color = "salmon"
    p3.title.text_font_size = "20px"
    p3.title.background_fill_color = "white"
    p3.title.text_font='helvetica'
    p3.title.text_alpha=0.8
    p3.xaxis.formatter = DatetimeTickFormatter(months="%B %Y", years="%B %Y")
    p3.xaxis.major_label_orientation = 3/4
    
    # Add HoverTool showing the exact lean mass index and measurement date values
    hover = HoverTool(tooltips=[('Lean Mass Index', '@{Lean Mass Index}'), 
                                ('Date', '@Date{%F}')], 
                      formatters={'@Date':'datetime'}, 
                      mode='vline')
    p3.add_tools(hover)
 
    # Plot change in sum of 7 skinfolds over time
    p4 = figure(plot_width=475, 
                plot_height=475, 
                x_axis_type='datetime', 
                x_axis_label='Measurement Date', 
                y_axis_label='Sum of 7 Skinfolds (mm)', 
                tools="", 
                toolbar_location=None)
    p4.line(x='Date', 
            y='Sum of 7', 
            line_width=2, 
            color='lightslategray', 
            alpha=0.5, 
            line_dash=[5,5], 
            source=data)
    p4.circle(x='Date', 
              y='Sum of 7', 
              size=15, 
              color='cadetblue', 
              alpha=0.8, 
              source=data)
    p4.title.text = f'Sum of 7 Skinfolds of Athlete {ID}'
    p4.title.align = "center"
    p4.title.text_color = "cadetblue"
    p4.title.text_font_size = "20px"
    p4.title.background_fill_color = "white"
    p4.title.text_font='helvetica'
    p4.title.text_alpha=0.8
    p4.xaxis.formatter = DatetimeTickFormatter(months="%B %Y", years="%B %Y")
    p4.xaxis.major_label_orientation = 3/4

    # Add HoverTool showing the exact sum of 7 and date values
    hover = HoverTool(tooltips=[('Sum of 7 Skinfolds', '@{Sum of 7}'), 
                                ('Date', '@Date{%F}')], 
                      formatters={'@Date':'datetime'}, 
                      mode='vline')
    p4.add_tools(hover)
    
    # Alter layout so graphs appear side by side in a row
    layout2 = row(p3, p4)
    output_notebook()
    show(layout2)

In [33]:
# Use for loop to generate graphs of all athletes using the athlete function
for i in range(len(anthro_data['ID'].unique())):
    athlete(anthro_data['ID'].unique()[i])

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...